#**Machine Learning Assignment 2 - Federated Learning**

submitted by :-

*   Praneet Karna
*   Jaysheel Shah
*   Nishal Shah








#Training using Federated Learning (10 clients)



Mounting google drive to get the data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importing required libraries - Numpy and Pandas

In [12]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import time
import os
from torchvision import datasets
import pandas as pd
from sklearn.metrics import confusion_matrix

Creating custom dataset class suited to our folder structure

In [18]:
class Dataset(Dataset):
    def __init__(self, directory, transform_function=None, target_size=(256,256)):
        self.directory = directory
        self.transform_function = transform_function
        self.target_size = target_size

        self.labels = [label for label in sorted(os.listdir(directory))
                             if os.path.isdir(os.path.join(directory, label))]
        self.label_to_index = {label: index for index, label in enumerate(self.labels)}

        # Build a list of images and corresponding labels
        self.samples = []
        for label in self.labels:
            path = os.path.join(directory, label)
            if os.path.isdir(path):
              for filename in os.listdir(path):
                filepath = os.path.join(path, filename)
                if os.path.isfile(filepath):
                  self.samples.append((filepath, self.label_to_index[label]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
      path, label = self.samples[index]
      try:
          image = Image.open(path).convert("RGB")
      except Exception as e:
          print(f"Error opening image at {path}: {e}")
          raise

      if self.transform_function:
          image = self.transform_function(image)

      return image, label


Creating a custom CNN network for animal image classification with 4 classes

In [4]:
class CustomCNN(nn.Module):
    def __init__(self, num_classes=4):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32 * 64 * 64, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        # print("After conv1:", x.shape)
        x = self.relu1(x)
        # print("After relu1:", x.shape)
        x = self.pool1(x)
        # print("After pool1:", x.shape)
        x = self.conv2(x)
        # print("After conv2:", x.shape)
        x = self.relu2(x)
        # print("After relu2:", x.shape)
        x = self.pool2(x)
        # print("After pool2:", x.shape)
        x = self.flatten(x)
        # print("After flatten:", x.shape)
        x = self.fc1(x)
        # print("After fc1:", x.shape)
        x = self.relu3(x)
        # print("After relu3:", x.shape)
        x = self.fc2(x)
        # print("After fc2:", x.shape)
        return x


Function to filter clients during each epoch based on metadata (battery life and signal strength).

Current requirements : 30% battery and above, 2/5 signal strength and above.

In [8]:
def approve_client(client_number, epoch_number):
  meta_data_path = '/content/drive/My Drive/DividedLabeledDataset/metadata.csv'
  meta_data = pd.read_csv(meta_data_path)
  index = epoch_number*10 + client_number - 1

  if(meta_data.at[index, 'battery life']>=30 and meta_data.at[index,'signal strength']>=2):
    return True
  return False

Function to get average of weights from a list of models

In [9]:
def get_average_of_models(models):
    n = len(models)
    average = models[0].state_dict()

    for i in range(1, n):
        current = models[i].state_dict()
        average = {x: average[x] + current[x] for x in average}

    average = {x: parameter / n for x, parameter in average.items()}

    return average

Defining the folder names containing data for every epoch for every client.

Currently : 10 clients and 4 total epochs

In [10]:
folderNames_epoch1 =  ['Part_1', 'Part_2', 'Part_3', 'Part_4', 'Part_5', 'Part_6', 'Part_7', 'Part_8', 'Part_9', 'Part_10']
folderNames_epoch2 =  ['Part_11', 'Part_12', 'Part_13', 'Part_14', 'Part_15', 'Part_16', 'Part_17', 'Part_18', 'Part_19', 'Part_20']
folderNames_epoch3 =  ['Part_21', 'Part_22', 'Part_23', 'Part_24', 'Part_25', 'Part_26', 'Part_27', 'Part_28', 'Part_29', 'Part_30']
folderNames_epoch4 =  ['Part_31', 'Part_32', 'Part_33', 'Part_34', 'Part_35', 'Part_36', 'Part_37', 'Part_38', 'Part_39', 'Part_40']

folderNames_list = [folderNames_epoch1,folderNames_epoch2,folderNames_epoch3,folderNames_epoch4]

Main function where the federated learning is performed for 4 epochs

In [20]:
def perform_federated_learning(base_model, num_epochs, learning_rate, list_of_client_lists):

    for epoch in range(num_epochs):
      folder_name_list = []
      client_list = folderNames_list[epoch]

      # selecting clients
      for client in range(10):
        if (approve_client(client+1,epoch)):
          folder_name_list.append("/content/drive/My Drive/DividedLabeledDataset/" + folderNames_list[epoch][client])

      trained_client_models = []

      for folder in folder_name_list:
            print(f"Training started for {folder}");
            current_client_model = CustomCNN()
            current_client_model.load_state_dict(base_model.state_dict())
            current_client_model.train

            criterion = nn.CrossEntropyLoss()
            optimizer = optim.SGD(current_client_model.parameters(), lr=learning_rate)

            transform = transforms.Compose([
                transforms.Resize((256, 256)),
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ])

            training_data = Dataset(folder, transform_function=transform)
            training_data_loader = DataLoader(training_data, batch_size=32, shuffle=True)

            for inputs, labels in training_data_loader:
                outputs = current_client_model(inputs)
                loss = criterion(outputs, labels)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            trained_client_models.append(current_client_model)

      print(f"Training completed for epoch {epoch}")

      # averaging the weights of trained client models
      base_model.load_state_dict(get_average_of_models(trained_client_models))
      base_model.eval()

      transform = transforms.Compose([
                transforms.Resize((256, 256)),
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ])

      testing_data = Dataset("/content/drive/MyDrive/DividedLabeledDataset/test", transform_function=transform)
      testing_data_loader = DataLoader(testing_data, batch_size=32, shuffle=True)

      num_total_examples = 0
      num_correct_examples = 0

      with torch.no_grad():
        for inp, lbs in testing_data_loader:
          outputs = base_model(inp)
          _, predicted = torch.max(outputs.data, 1)
          num_correct_examples += (predicted == lbs).sum().item()
          num_total_examples += lbs.size(0)

      testing_accuracy = num_correct_examples / num_total_examples
      print(f'Testing Accuracy for epoch {epoch}: {100 * testing_accuracy:.2f}%')

      # Print Confusion Matrix
      print("Confusion Matrix:")
      with torch.no_grad():
          all_labels = []
          all_predictions = []
          for inputs, labels in testing_data_loader:
              outputs = base_model(inputs)
              _, predicted = torch.max(outputs.data, 1)

              all_labels.extend(labels.numpy())
              all_predictions.extend(predicted.numpy())

          cm = confusion_matrix(all_labels, all_predictions)
          print(cm)

      print(f"Epoch {epoch + 1}/{num_epochs} completed")

Running the federated learning function on a new base model

In [21]:
base_model = CustomCNN()
perform_federated_learning(base_model, num_epochs=4, learning_rate=0.001, list_of_client_lists=folderNames_list)

Training started for /content/drive/My Drive/DividedLabeledDataset/Part_1
Training started for /content/drive/My Drive/DividedLabeledDataset/Part_3
Training started for /content/drive/My Drive/DividedLabeledDataset/Part_5
Training started for /content/drive/My Drive/DividedLabeledDataset/Part_6
Training started for /content/drive/My Drive/DividedLabeledDataset/Part_8
Training started for /content/drive/My Drive/DividedLabeledDataset/Part_9
Training started for /content/drive/My Drive/DividedLabeledDataset/Part_10
Training completed for epoch 0
Testing Accuracy for epoch 0: 20.96%
Confusion Matrix:
[[ 65 135   0   0]
 [ 31  84   0   0]
 [ 46 187   0   0]
 [ 35 128   0   0]]
Epoch 1/4 completed
Training started for /content/drive/My Drive/DividedLabeledDataset/Part_11
Training started for /content/drive/My Drive/DividedLabeledDataset/Part_13
Training started for /content/drive/My Drive/DividedLabeledDataset/Part_14
Training started for /content/drive/My Drive/DividedLabeledDataset/Part_1

## Training without using Federated Learning

Training the model on the same training data

In [22]:
model = CustomCNN()
model.train
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
          ])

training_dataset = Dataset("/content/drive/MyDrive/imge-classification-animal/animal-classification/train", transform_function=transform)
training_dataset_loader = DataLoader(training_dataset, batch_size=32, shuffle=True)

print(f"Training started.")

for inp, lbs in training_dataset_loader:
  outputs = model(inp)
  loss = criterion(outputs, lbs)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

model.eval()
print(f"Training ended.")

Training started.
Training ended.


Checking accuracy of model trained without federated learning

In [23]:
testing_data = Dataset("/content/drive/MyDrive/DividedLabeledDataset/test", transform_function=transform)
testing_data_loader = DataLoader(testing_data, batch_size=32, shuffle=True)

num_total_examples = 0
num_correct_examples = 0

with torch.no_grad():
  for inp, lbs in testing_data_loader:
    outputs = model(inp)
    x, predicted = torch.max(outputs.data, 1)
    num_correct_examples += (predicted == lbs).sum().item()
    num_total_examples += lbs.size(0)

testing_accuracy = num_correct_examples / num_total_examples
# printing the training accuracy of this epoch of federated learning
print(f'Testing Accuracy without federated learning: {100 * testing_accuracy:.2f}%')

Testing Accuracy without federated learning: 32.77%
